# Imports

In [ ]:
import json

from algosdk.v2client import algod

from setup.create_app import create_app
from setup.opt_in_app import opt_in_app
from setup.call_app import call_app
from setup.delete_app import delete_app
from setup.update_app import update_app


from setup.helper_functions import *

from setup.pool_functions.opt_into_asset import opt_into_asset
from setup.pool_functions.rekey_to_application import rekey_pool_to_application
from setup.pool_functions.create_asset import create_asset



# Initializing Algod Client

In [ ]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# initialize an algodClient
algod_client = algod.AlgodClient(algod_token, algod_address)

# Obtaining data from keys.json

In [ ]:
# open json file with address and mnemonic data
f = open('keys.json')
data = json.load(f)



# Creator address and private key. 
# This account is a multisig (1/2). 
# For developement purposes, both private keys are stored in keys.json
# This will change in a production environment for security purposes. 
creator_mnemonic_1 = data['creator_multisig_accounts']['creator_mnemonic_1']
creator_private_key_1 = get_private_key_from_mnemonic(creator_mnemonic_1)
creator_address = data['creator_multisig_address']


# Lender USDC Deposit Pool.
# This account is single sig Algorand account.
# The private key and address is stored in keys.json for development purposes
# This account will be rekeyed to the smart contract
lender_usdc_pool_mnemonic = data['lender_usdc_pool_mnemonic']
lender_usdc_pool_private_key = get_private_key_from_mnemonic(lender_usdc_pool_mnemonic)
lender_usdc_pool_address = data['lender_usdc_pool_address']


# Index Token Reserve Pool.
# This account is single sig Algorand account.
# The private key and address is stored in keys.json for development purposes
# This account will be rekeyed to the smart contract
token_reserve_pool_mnemonic = data['token_reserve_pool_mnemonic']
token_reserve_pool_private_key = get_private_key_from_mnemonic(token_reserve_pool_mnemonic)
token_reserve_pool_address = data['token_reserve_pool_address']



# Index Token Active Pool.
# This account is single sig Algorand account.
# The private key and address is stored in keys.json for development purposes
# This account will be rekeyed to the smart contract
token_active_pool_mnemonic = data['token_active_pool_mnemonic']
token_active_pool_private_key = get_private_key_from_mnemonic(token_active_pool_mnemonic)
token_active_pool_address = data['token_active_pool_address']



# Index Token Stake Pool.
# This account is single sig Algorand account.
# The private key and address is stored in keys.json for development purposes
# This account will be rekeyed to the smart contract
token_stake_pool_mnemonic = data['token_stake_pool_mnemonic']
token_stake_pool_private_key = get_private_key_from_mnemonic(token_stake_pool_mnemonic)
token_stake_pool_address = data['token_stake_pool_address']



# Payback Pool.
# This account is single sig Algorand account.
# The private key and address is stored in keys.json for development purposes
# This account will be rekeyed to the smart contract
payback_pool_mnemonic = data['payback_pool_mnemonic']
payback_pool_private_key = get_private_key_from_mnemonic(payback_pool_mnemonic)
payback_pool_address = data['payback_pool_address']





# User private key. 
# In this case it is a singlesig, but it could also be a multisig if the user wants to make it one. 
# For developement purposes, this private key is stored in keys.json
user_mnemonic = data['user_mnemonic']
user_private_key = get_private_key_from_mnemonic(user_mnemonic)
user_address = data['user_mnemonic']

# USDC ID
usdc_id = data['usdc_id']
index_token_id = data['index_token_id']

app_id = data['app_id']
application_address = data['app_address']


# Closing json file
f.close()

# Creating Application

In [ ]:
'''
app_id, application_address = create_app(algod_client, creator_address, creator_private_key_1)
print("Created new app-id:", app_id)
print("Application address:", application_address)

# read global state of application
print( "Global state:", read_global_state(algod_client, creator_address, app_id) )

# read local state of application from user account
print( "Local state:", read_local_state(algod_client, creator_address, app_id) )
'''



# Update Application

In [ ]:
update_app(algod_client, creator_address, creator_private_key_1, app_id)

# read global state of application
print( "Global state:", read_global_state(algod_client, creator_address, app_id) )

# read local state of application from user account
print( "Local state:", read_local_state(algod_client, creator_address, app_id) )

##### --------------------------------------------------------------------------------------------
# **lender_usdc_pool**: 3 actions

##### 1) ----------> Opt pool into smart contract 
##### 2) ----------> Opt pool into USDC 
##### 3) ----------> Rekey pool to smart contract

##### --------------------------------------------------------------------------------------------


##### Action 1: Opt Lender USDC Pool into Smart Contract

In [ ]:
# if pool account is not opted into the app, opt into it
if not is_opted_in_app(algod_client, lender_usdc_pool_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, lender_usdc_pool_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)
    

##### Action 2: Opt Lender USDC Pool into USDC

In [ ]:
# if pool is not opted into the asset, opt into it
if not is_opted_in_asset(algod_client, usdc_id, lender_usdc_pool_address):

    print("Account is not opted in to USDC.. opting in now...")

    # opt into USDC from pool account
    result = opt_into_asset(algod_client, lender_usdc_pool_address, lender_usdc_pool_private_key, usdc_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

# print pool asset holdings
print_asset_holding(algod_client, lender_usdc_pool_address, usdc_id)


##### Action 3: Rekey Lender USDC Pool to Smart Contract

In [ ]:
if not is_application_address(algod_client, application_address, lender_usdc_pool_address): 
    
    result = rekey_pool_to_application(algod_client, application_address, lender_usdc_pool_address, lender_usdc_pool_private_key)

    # print result confirmation transaction confirmation 
    print_confirmation(result)


##### --------------------------------------------------------------------------------------------
# **token_reserve_pool**: 3 actions

##### 1) ----------> Opt pool into smart contract 
##### 2) ----------> Create index_token (infinite amount)
##### 3) ----------> Rekey pool to smart contract

##### --------------------------------------------------------------------------------------------

##### Action 1: Opt reserve pool into Smart Contract

In [ ]:
# if pool account is not opted into the app, opt into it
if not is_opted_in_app(algod_client, token_reserve_pool_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, token_reserve_pool_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)



##### Action 2: Create index token and store in reserve address

In [ ]:
if not is_opted_in_asset(algod_client, index_token_id, token_reserve_pool_address):
    
    create_asset(algod_client, creator_address, token_reserve_pool_address, token_reserve_pool_private_key)

##### Action 3: Rekey reserve token pool to smart contract

In [ ]:
if not is_application_address(algod_client, application_address, token_reserve_pool_address): 
    
    result = rekey_pool_to_application(algod_client, application_address, token_reserve_pool_address, token_reserve_pool_private_key)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

##### --------------------------------------------------------------------------------------------
# **token_active_pool**: 3 actions

##### 1) ----------> Opt pool into smart contract 
##### 2) ----------> Opt pool into index_token
##### 3) ----------> Rekey pool to smart contract

##### --------------------------------------------------------------------------------------------

##### Action 1: Opt active pool into Smart Contract

In [ ]:
# if pool account is not opted into the app, opt into it
if not is_opted_in_app(algod_client, token_active_pool_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, token_active_pool_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

##### Action 2: Opt active pool into index_token (LCASA)

In [ ]:
# if pool is not opted into the asset, opt into it
if not is_opted_in_asset(algod_client, index_token_id, token_active_pool_address):

    print("Account is not opted in to index token (LCASA).. opting in now...")

    # opt into USDC from pool account
    result = opt_into_asset(algod_client, token_active_pool_address, token_active_pool_private_key, index_token_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

# print pool asset holdings
print_asset_holding(algod_client, token_active_pool_address, index_token_id)

##### Action 3: Rekey active token pool to smart contract

In [ ]:
if not is_application_address(algod_client, application_address, token_active_pool_address): 
    
    result = rekey_pool_to_application(algod_client, application_address, token_active_pool_address, token_active_pool_private_key)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

##### --------------------------------------------------------------------------------------------
# **token_stake_pool**: 3 actions

##### 1) ----------> Opt pool into smart contract 
##### 2) ----------> Opt pool into index_token
##### 3) ----------> Rekey pool to smart contract

##### --------------------------------------------------------------------------------------------

##### Action 1: Opt stake pool into Smart Contract

In [ ]:
# if pool account is not opted into the app, opt into it
if not is_opted_in_app(algod_client, token_stake_pool_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, token_stake_pool_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)


##### Action 2: Opt stake pool into index_token (LCASA)

In [ ]:
# if pool is not opted into the asset, opt into it
if not is_opted_in_asset(algod_client, index_token_id, token_stake_pool_address):

    print("Account is not opted in to index token (LCASA).. opting in now...")

    # opt into USDC from pool account
    result = opt_into_asset(algod_client, token_stake_pool_address, token_stake_pool_private_key, index_token_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

# print pool asset holdings
print_asset_holding(algod_client, token_stake_pool_address, index_token_id)

##### Action 3: Rekey stake token pool to smart contract

In [ ]:
if not is_application_address(algod_client, application_address, token_stake_pool_address): 
    
    result = rekey_pool_to_application(algod_client, application_address, token_stake_pool_address, token_stake_pool_private_key)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

##### --------------------------------------------------------------------------------------------
# **payback_pool**: 3 actions

##### 1) ----------> Opt pool into smart contract 
##### 2) ----------> Opt pool into USDC
##### 3) ----------> Rekey pool to smart contract

##### --------------------------------------------------------------------------------------------

##### Action 1: Opt Payback Pool into Smart Contract

In [ ]:
# if pool account is not opted into the app, opt into it
if not is_opted_in_app(algod_client, payback_pool_address, app_id):

    # opt into application from pool account
    result = opt_in_app(algod_client, payback_pool_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

##### Action 2: Opt Payback Pool into USDC

In [ ]:
# if pool is not opted into the asset, opt into it
if not is_opted_in_asset(algod_client, usdc_id, payback_pool_address):

    print("Account is not opted in to USDC.. opting in now...")

    # opt into USDC from pool account
    result = opt_into_asset(algod_client, payback_pool_address, payback_pool_private_key, usdc_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

# print pool asset holdings
print_asset_holding(algod_client, payback_pool_address, usdc_id)

##### Action 3: Rekey Payback Pool to Smart Contract

In [ ]:
if not is_application_address(algod_client, application_address, payback_pool_address): 
    
    result = rekey_pool_to_application(algod_client, application_address, payback_pool_address, payback_pool_private_key)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

# User OptIn to Application

In [ ]:
# if account is not opted into the app, opt in
if not is_opted_in_app(algod_client, user_address, app_id):

    # opt into application from user account
    result = opt_in_app(algod_client, user_private_key, app_id)

    # print result confirmation transaction confirmation 
    print_confirmation(result)

    # print logs
    print_logs(result)


# read global state of application
print( "Global state:", read_global_state( algod_client, creator_address, app_id) )

# read local state of application from user account
print( "Local state:", read_local_state( algod_client, user_address, app_id) )

# Calling Application

In [ ]:
args = [b"vote", b"choiceA"]

if not is_opted_in_asset: 
    # opt in to asset function 
    pass
    
result = call_app(algod_client, user_private_key, app_id, args)    


# read local state of application from user account
print( "Local state:", read_local_state(algod_client, user_address, app_id) )

# read global state of application
print( "Global state:", read_global_state(algod_client, creator_address, app_id) )